In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [3]:
#Read datasets
uk_accidents = pd.read_csv('data/uk_accidents/Accident_Information.tar.xz', low_memory=False)

In [4]:

# Convert Long Lat into numeric type
uk_accidents['Longitude'] = pd.to_numeric(uk_accidents['Longitude'])
uk_accidents['Latitude'] = pd.to_numeric(uk_accidents['Latitude'])


#  Convert Longitude Latitude into Point Geometry
uk_accidents = gpd.GeoDataFrame(uk_accidents, geometry = gpd.points_from_xy(x=uk_accidents['Longitude'], y=uk_accidents['Latitude']))
uk_accidents = uk_accidents.set_crs('EPSG:4326')

# Remove invalid geometries if any
uk_accidents = uk_accidents[uk_accidents.is_valid]
uk_accidents = uk_accidents[~uk_accidents.is_empty]
uk_accidents = uk_accidents.to_crs('EPSG:3857')


In [6]:
# Perform spatial queries, such as finding accidents within a certain distance from a specific location:

# Creating a sample point
point = Point((-13411.1, 6711284.0))

# Creating a buffer of 1 KM around the point
buffer = point.buffer(1000)

within_buffer = uk_accidents[uk_accidents.within(buffer)]


In [9]:
# Load the administrative regions shapefile
uk_admins = gpd.read_file('data/uk_admin_region/uk_admins_region.shp')
uk_admins = uk_admins.to_crs('EPSG:3857')

# Perform a spatial join between the accidents GeoDataFrame and the administrative regions shapefile
merged = gpd.sjoin(uk_accidents, uk_admins, how="inner", predicate='within')

# Group the merged GeoDataFrame by the administrative region and count the number of accidents in each region
grouped = merged.groupby('ADMIN_REGI').count()['Accident_Index']


In [12]:
merged[['Accident_Index', 'Accident_Severity', 'ADMIN_REGI']]

,Accident_Index,Accident_Severity,ADMIN_REGI
0,200501BS00001,Serious,Kensington and Chelsea
1,200501BS00002,Slight,Kensington and Chelsea
3,200501BS00004,Slight,Kensington and Chelsea
5,200501BS00006,Slight,Kensington and Chelsea
6,200501BS00007,Slight,Kensington and Chelsea
...,...,...,...
2047015,201797UD71011,Serious,South Ayrshire
2047016,201797UD71101,Slight,South Ayrshire
2047017,201797UD71103,Serious,South Ayrshire
2047018,201797UD71106,Slight,South Ayrshire


In [13]:
grouped

ADMIN_REGI
Aberdeen            4476
Aberdeenshire       6503
Anglesey            1285
Angus               2492
Argyll and Bute     3147
                   ...  
Westminster        20826
Wiltshire          14410
Worcestershire     16784
Wrexham             3999
York                6217
Name: Accident_Index, Length: 166, dtype: int64